# Infrastructure Agent: Distributed Architecture

The same agent capabilities as V5 (The Full Agent), but deployed as distributed services.
Direct Python tool imports are replaced by MCP server connections, and in-process sub-agents
are replaced by remote A2A servers.

What changes:
- Tools come from MCP servers (file_ops, sandbox, todo, format_conversion)
- Delegation goes to A2A servers (nl2sql, data_analysis, vocabulary)

What stays the same:
- Prompt structure and skill discovery
- AgentSpec composition pattern
- OrchestratorAgent run loop

**Prerequisite:** Start all servers with `scripts/launch_infrastructure.sh`, then run
`scripts/db_ingest_bookstore_sqlite.sh` to create the bookstore database.

In [ ]:
from agentic_patterns.core.agents import AgentSpec, OrchestratorAgent
from agentic_patterns.core.a2a import get_a2a_client
from agentic_patterns.core.config.config import PROMPTS_DIR, SKILLS_DIR
from agentic_patterns.core.mcp import load_mcp_settings, MCPClientConfig
from agentic_patterns.core.workspace import clean_up_session

## Agent Definition

The infrastructure agent uses `mcp_servers` for tools and `a2a_clients` for delegation,
instead of direct Python tool imports and in-process sub-agents. No `tools` or `sub_agents`
fields are needed -- everything comes from the network.

In [ ]:
# Load MCP client configs from config.yaml
mcp_settings = load_mcp_settings()
mcp_names = ["file_ops", "sandbox", "todo", "format_conversion"]
mcp_servers = [
    config for name in mcp_names
    if isinstance((config := mcp_settings.get(name)), MCPClientConfig)
]

# Load A2A clients from config.yaml
a2a_names = ["nl2sql", "data_analysis", "vocabulary"]
a2a_clients = [get_a2a_client(name) for name in a2a_names]

spec = AgentSpec(
    name="infrastructure_agent",
    system_prompt_path=PROMPTS_DIR / "the_complete_agent" / "agent_infrastructure.md",
    mcp_servers=mcp_servers,
    a2a_clients=a2a_clients,
    # Skills auto-discovered from SKILLS_DIR
)
print(spec)

In [ ]:
clean_up_session()
agent = OrchestratorAgent(spec, verbose=True)

## Turn 1: Synchronous Delegation

Ask about data in the bookstore database. The agent delegates to the nl2sql A2A server,
which connects to the SQL MCP server.

In [ ]:
async with agent:
    prompt_1 = """Query the bookstore database: how many books are there,
what genres are represented, and what is the average price per genre?
Save the per-genre results to a CSV file."""

    result_1 = await agent.run(prompt_1)

    print("\n--- Agent Output ---")
    print(result_1.output)

## Turn 2: Parallel Delegation

Ask the agent to do multiple independent tasks. It delegates to nl2sql and data_analysis
A2A servers, then assembles a report.

In [ ]:
async with agent:
    prompt_2 = """I need two things:
1. Query the bookstore database for the top 5 most expensive books with their authors and genres.
2. Create a bar chart showing the average price per genre (save as /workspace/avg_price_by_genre.png).

Once both are done, write a markdown report at /workspace/bookstore_report.md combining all findings."""

    result_2 = await agent.run(prompt_2)

    print("\n--- Agent Output ---")
    print(result_2.output)

## Verify Workspace

In [ ]:
from agentic_patterns.core.workspace import list_workspace_files, read_from_workspace

for path in sorted(list_workspace_files("*")):
    try:
        content = read_from_workspace(path)
    except UnicodeDecodeError:
        content = "<binary file>"
    print(f"--- {path} ---")
    print(content)
    print()